# Calibration of ADCs

In [1]:
import sys
import numpy as np
import pyfftw
import pickle

import ROOT
from ROOT import gROOT
from ROOT import gStyle

%jsroot on

from array import array
#--------------------------------------------

gROOT.LoadMacro("../style/CMS/tdrstyle.C")
gROOT.ProcessLine("setTDRStyle();")
gROOT.LoadMacro("../style/CMS/CMS_lumi.C")


ROOT.gInterpreter.AddIncludePath('../../include/')
ROOT.gInterpreter.Declare('#include "RoEvent.h"')
ROOT.gSystem.Load('../../libs/libRoEvent.so')

gStyle.SetOptFit(0)

Welcome to JupyROOT 6.24/00


In [2]:
class Signal(object):
    
    ev = 0
    ch = 0
    
    peak_index = 0
    peak_value = 0
    
    polarity = 0
    threshold = 0 
    
    width = 0
    integral = 0
    
    #analysis
    unequal = 0
    isFFT = 0 # pedestal correction 
    
    pedestal = []
    data = []
    raw_data = []

In [3]:
# Remove Pedestal
def correctPedestal(signal, pedestal):
    s = pyfftw.interfaces.numpy_fft.fft(signal)
    ss = np.abs(s)
    angle = np.angle(s)
    b = np.exp(1.0j* angle)

    ns= pyfftw.interfaces.numpy_fft.fft(pedestal)
    nss = np.abs(ns)
    mns = np.mean(nss)

    sa= ss - nss
    sa0= sa * b  # apply phase information

    result = pyfftw.interfaces.numpy_fft.ifft(sa0).real
    return result

In [4]:
# Remove Pedestal
def subPedestal(data, pol):
    fft = pyfftw.interfaces.numpy_fft.fft(data)
    fft[0] = 0 
    ifft = pyfftw.interfaces.numpy_fft.ifft(fft).real
    idata = array('f')
    for i in range(len(ifft)):
        idata.append(pol*round(ifft[i]));
    return idata

In [5]:
#get signals from array
from scipy.signal import find_peaks,peak_widths
def findSignals(data, ev = 0, ch = 0, thr=50, pad=30, pol = 1):
    output = []
    idata = subPedestal(data, pol)
    peaks, _ = find_peaks(idata, height=thr)
    for p in range(len(peaks)):
        sig = Signal()
        sig.isFFT = 1
        sig.ev = ev 
        sig.ch = ch
        sig.peak_index = peaks[p]
        sig.peak_value = idata[peaks[p]]
        sig.threshold = thr
        sig.polarity = pol
        sig.width = peak_widths(idata, peaks, rel_height=0.2)[0][p]
        sig.integral = sum(idata[peaks[p] - pad: peaks[p] + pad])
        sig.data = idata[peaks[p] - pad: peaks[p] + pad]
        p_peaks, _ = find_peaks(idata[peaks[p] - pad - 2*pad : peaks[p] - pad], height=thr)
        if(len(p_peaks) == 0):
            sig.pedestal = idata[peaks[p] - pad - 2*pad : peaks[p] - pad]
        sig.raw_data = data[0:len(data)]
        output.append(sig)
        
    return output

In [6]:
run = "r602"
thr = 70; pad = 30
root = "../../tmp/ro_" + run + ".root"
pick = "../../tmp/ju_" + run + "_thr-" + str(thr) + "_pad-" + str(pad) + ".pik"
print("ROOT file: {}".format(root))
print("PICK file: {}".format(pick))

#get data from RoEvent class (to generate it use dq script)
f = ROOT.TFile(root)
myTree = f.Get("data")
#myTree.Print()
myTree.Show(560)

ROOT file: ../../tmp/ro_r602.root
PICK file: ../../tmp/ju_r602_thr-70_pad-30.pik
======> EVENT:560
 events          = (RoEvent*)0x7f5030611420
 fUniqueID       = 0
 fBits           = 33554432
 fEvtHdr.fUniqueID = 0
 fEvtHdr.fBits   = 33554432
 fEvtHdr.fEvtNum = 561
 fEvtHdr.fValid  = 1
 fEvtHdr.fGood   = 0
 fEvtHdr.fFlags[8] = 0 , 37888 , 50783 , 32767 , 0 , 1 , 0 , 0 

 fEvtHdr.fRunNum = 602
 fEvtHdr.fRunFlags = 32767
 fEvtHdr.fTrigType = 0
 fEvtHdr.fTrigNum = 561
 fEvtHdr.fTvNsec = 275569200
 fNumPayloads    = 128
 fPayloads       = 128
 fPayloads.fUniqueID = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
 fPayloads.fBits = 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648
 fPayloads.fCh   = 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9
 fPayloads.fMap.fUniqueID = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [7]:
#loop over chs
ro_signals = []
for entry in myTree:
    events = entry.events
    numCh = events.getNumPayloads()
    ev = events.getHeader().getEvtNum()
    print("Ev {} is processing... ".format(ev), end='\r')
    for n in range(numCh):
        ch = events.getPayloads().AddrAt(n).getCh()
        cro = events.getPayloads().AddrAt(n).getCRO()
        numSa = events.getPayloads().AddrAt(n).getNsaCRO()
        data = array('i')
        for ns in range(numSa): 
            data.append(int(cro[ns]));
        ev_signals_pos = findSignals(data, ev=ev, ch=ch, thr=thr, pad=pad, pol=1)
        ev_signals_neg = findSignals(data, ev=ev, ch=ch, thr=thr, pad=pad, pol=-1)
        ro_signals.extend(ev_signals_pos)
        ro_signals.extend(ev_signals_neg)

with open(pick, "wb") as f:
    pickle.dump(len(ro_signals), f)
    for value in ro_signals:
        pickle.dump(value, f)

KeyboardInterrupt: 

In [35]:
# read ju_signals
ju_signals = []
with open(pick, "rb") as f:
    for _ in range(pickle.load(f)):
        ju_signals.append(pickle.load(f))  
print(len(ju_signals))

#chs = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25,26,27,28,29,30,31]
chs = range(32, 63)
profiles = []
anomalies = []
chs_thr = []
pol = 1; N = 120
for ch in chs:  
    profile = ROOT.TH2D("profile_" + str(ch), "profile_" + str(ch), 2*pad, 0, 2*pad - 1, 4096, -2048, 2048);
    for sig in ju_signals:
        if(pol == sig.polarity and (sig.ch == ch)):
            if(len(sig.pedestal) > 0 and len(sig.data) == 2*pad):
                data = correctPedestal(sig.data, sig.pedestal)
                for s in range(len(data)):
                    profile.Fill(s, pol*data[s])
    profiles.append(profile)
    xmin = sys.float_info.max
    xmax = sys.float_info.min
    anomaly = ROOT.TH1D("anomaly_" + str(ch), "anomaly_" + str(ch), 300, 1, -1);
    for sig in ju_signals:
        if(pol == sig.polarity and (sig.ch == ch)):
            value = 0
            for s in range(len(sig.data)):
                if(sig.data[s] != 0):
                    value += profile.GetBinContent(int(s), int(sig.data[s]) + 2048);
            if(xmin > value):
                xmin = value
            if(xmax < value):
                xmax = value
            anomaly.Fill(value)
            sig.unequal = value
    anomalies.append(anomaly)
    step = 1
    for thr in range(int(xmin), int(xmax), step):
        sig_N = 0;
        for sig in ju_signals:
            if(sig.unequal > thr and (sig.ch == ch)):
                if(len(sig.pedestal) > 0 and len(sig.data) == 2*pad):
                    sig_N += 1
        print("ch={} n = {}".format(ch, sig_N), end="\r")
        if(sig_N < N):
            chs_thr.append(thr - step)
            break;
print(chs_thr)

10252
[138, 93, 217, 159, 157, 153, 160, 154, 188, 156, 482, 585, 406, 263, 156, 186, 157, 243, 248, 165, 404, 450, 154, 194, 678, 171, 164, 153, 151, 160, 164]


In [36]:
# new profile of ch & average signal
profile_all = ROOT.TH2D("filtered_profile_all", "filtered_profile_all", 2*pad, 0, 2*pad - 1, 4096, -2048, 2048);
profiles_f = []
average_all = []
average = []
height = []
for i in range(len(chs)):
    h_sum = ROOT.TH1D("sum_" + str(i), "sum_" + str(i), 30, 1, -1);
    average.append(h_sum);
    h_h = ROOT.TH1D("height_" + str(i), "height_" + str(i), 30, 1, -1);
    height.append(h_h);
for i in range(2*pad):
    average_all.append([]); 
sig_N = 0
for i in range(len(chs)): 
    profile = ROOT.TH2D("filtered_profile_" + str(chs[i]), "filtered_profile_" + str(chs[i]), 2*pad, 0, 2*pad - 1, 4096, -2048, 2048);
    for sig in ju_signals:
        if(sig.unequal > chs_thr[i] and sig.ch == chs[i]):
            if(len(sig.pedestal) > 0 and len(sig.data) == 2*pad):
                sig_N += 1
                data = correctPedestal(sig.data, sig.pedestal)
                average[i].Fill(sum(data));
                peaks, _ = find_peaks(data, height=50)
                if(len(peaks)):
                    height[i].Fill(data[peaks[0]]);
                for s in range(len(data)):
                    average_all[s].append(data[s])
                    profile.Fill(s, pol*data[s])
                    profile_all.Fill(s, pol*data[s])
                profiles_f.append(profile)
print(sig_N)
print(sig_N/120)
print(len(chs))

sig_avg = array('f'); sig_std = array('f')
tick = array('f'); eTick = array('f')   

for i in range(len(average_all)):
    sig_avg.append(np.average(average_all[i]))
    sig_std.append(np.std(average_all[i]))
    tick.append(i)
    eTick.append(0)
    
gr = ROOT.TGraphErrors( len(sig_avg), tick, sig_avg, eTick, sig_std )
name = "average"; gr.SetTitle(name); gr.SetName(name)
gr.GetXaxis().SetTitle("tick"); gr.GetYaxis().SetTitle("ADCu")

d_sum = array('f'); d_sum_std = array('f')
d_ch = array('f'); err = array('f')
d_height = array('f'); d_height_std = array('f');

for i in range(len(chs)):
    d_sum.append(average[i].GetMean())
    d_sum_std.append(average[i].GetRMS())
    
    d_height.append(height[i].GetMean())
    d_height_std.append(height[i].GetRMS());
    
    d_ch.append(chs[i] -32)
    err.append(0)
    
gr_h = ROOT.TGraphErrors( len(d_ch), d_ch, d_height, err, d_height_std )
name = ""; gr_h.SetTitle(name); gr_h.SetName(name)
gr_h.GetXaxis().SetTitle("chid"); gr_h.GetYaxis().SetTitle("sum")

gr_sum = ROOT.TGraphErrors( len(d_ch), d_ch, d_sum, err, d_sum_std )
name = ""; gr_sum.SetTitle(name); gr_sum.SetName(name)
gr_sum.GetXaxis().SetTitle("chid"); gr_sum.GetYaxis().SetTitle("height")


3735
31.125
31


Warning in <TROOT::Append>: Replacing existing TH1: filtered_profile_all (Potential memory leak).


In [18]:
print(sum(sig_avg))
peaks, _ = find_peaks(sig_avg, height=50)
print(sig_avg[peaks[0]])

812.7102681361139
80.35186767578125


In [37]:
print(np.mean(d_height))

print(np.std(d_height))

98.29661
8.03783


In [1]:
plot = profile_all.Clone()
#plot = profiles_f[0].Clone()
#plot = anomaly.Clone()
pathToSave = "/mnt/d/tmp/"
#---
iPeriod = "0"; iPos="0"
#---
gROOT.ProcessLine("cmsText = \"\";")
gROOT.ProcessLine("writeExtraText = true;") # if extra text
#gROOT.ProcessLine("writeExtraText = false;") #remove Preliminary
gROOT.ProcessLine("extraText  = \"\";") # default extra text is "Preliminary"
gROOT.ProcessLine("lumi_sqrtS = \" #bf{negative (thr:70ADCu)}  | not 2-4 & 18-19 | N(3240) = 3254 | run:601 12-05-21               \";") 


plot.GetYaxis().SetRangeUser(-200, 200);

#plot.SetMarkerColor(1); plot.SetMarkerStyle(20); plot.SetMarkerSize(0.2)
plot.SetLineColor(1); plot.GetXaxis().SetTitle("tick"); plot.GetYaxis().SetTitle("ADCu")
plot.SetFillStyle( 3001); plot.SetFillColor(4);


cName = "profile_filtered_neg_r602"; cTitle = "c"; cX = 800; cY = 600
c = ROOT.TCanvas(cName, cTitle, cX, cY)
#c.SetGrid()
#---

p1 = ROOT.TPad("p1", "", 0, 0, 1, 1);
p1.SetGrid();

# Draw the 1st TMultiGraph
p1.SetBottomMargin(0.2)
p1.SetRightMargin(0.15)
p1.Draw();
p1.cd();
plot.Draw("colz"); 
#line.Draw();
#fit_s.Draw()
cms_lumi_str = "CMS_lumi("+cName+", "+iPeriod+", "+iPos+");"; gROOT.ProcessLine(cms_lumi_str)

c.SaveAs("/mnt/d/" + cName + ".svg");
c.SaveAs("/mnt/d/" + cName + ".pdf");
plot.SaveAs("/mnt/d/" + cName + ".root");
c.Draw()


#--- 

NameError: name 'profile_all' is not defined

In [39]:
plot = gr.Clone()
plot = gr_h.Clone()
#plot = gr_sum.Clone()

pathToSave = "/mnt/d/tmp/"
#---
iPeriod = "0"; iPos="0"
#---
gROOT.ProcessLine("cmsText = \"\";")
gROOT.ProcessLine("writeExtraText = true;") # if extra text
#gROOT.ProcessLine("writeExtraText = false;") #remove Preliminary
gROOT.ProcessLine("extraText  = \"\";") # default extra text is "Preliminary"
gROOT.ProcessLine("lumi_sqrtS = \" #bf{pos (thr:70ADCu) height (mean: 98.3 std: 8.0)} | chs = all | N (3840) = 3735 | run:602\";") 

plot.SetMarkerColor(1); plot.SetMarkerStyle(20); plot.SetMarkerSize(0.5)
plot.SetLineColor(1); plot.GetXaxisк().SetTitle("chid"); plot.GetYaxis().SetTitle("ADCu")
plot.SetFillStyle( 3001); plot.SetFillColor(4);

plot.GetYaxis().SetRangeUser(-50, 300);
plot.GetXaxis().SetRangeUser(-1, 31);

cName = "height_pos_r602_same"; cTitle = "c"; cX = 800; cY = 600
c = ROOT.TCanvas(cName, cTitle, cX, cY)
#c.SetGrid()
#---

p1 = ROOT.TPad("p1", "", 0, 0, 1, 1);
p1.SetGrid();

# Draw the 1st TMultiGraph
p1.SetBottomMargin(0.2)
#p1.SetRightMargin(0.15)
p1.Draw();
p1.cd();
plot.Draw("AP"); 
#line.Draw();
#fit_s.Draw()
cms_lumi_str = "CMS_lumi("+cName+", "+iPeriod+", "+iPos+");"; gROOT.ProcessLine(cms_lumi_str)

c.SaveAs("/mnt/d/" + cName + ".svg");
c.SaveAs("/mnt/d/" + cName + ".pdf");
plot.SaveAs("/mnt/d/" + cName + ".root");
c.Draw()


#--- 

 #bf{pos (thr:70ADCu) height (mean: 98.3 std: 8.0)} | chs = all | N (3840) = 3735 | run:602


Warning in <TCanvas::Constructor>: Deleting canvas with same name: height_pos_r602_same
Info in <TCanvas::Print>: SVG file /mnt/d/height_pos_r602_same.svg has been created
Info in <TCanvas::Print>: pdf file /mnt/d/height_pos_r602_same.pdf has been created
Info in <TGraphErrors::SaveAs>: ROOT file /mnt/d/height_pos_r602_same.root has been created
